# Tutorial 1: QC report for ecg data

This tutorial demonstrates the QC report capability of *niphlem*. For simplicity, we focus on the cardiac signal from ecg for one subject from one of our internal projects. 

Define filepaths for physiological data (ECG and info logs):

In [4]:
info_log = "./data/demo/physio/Physio_20210322_140315_89a222d1-4c24-4caf-a898-f06c6bfd2342_Info.log"
ecg_log = "./data/demo/physio/Physio_20210322_140315_89a222d1-4c24-4caf-a898-f06c6bfd2342_ECG.log"

### Load data

First load the info file, which provides scanner timing using the  `niphlem.input_data.load_cmrr_info` function. 

In [9]:
from niphlem.input_data import load_cmrr_info

print(load_cmrr_info.__doc__)


    Load information log files from CMRR sequences.

    Parameters
    ----------
    filename : str, pathlike
        Path to Information Log file.

    Returns
    -------
    traces : ndarray
        Time ticks of the scanner.
    meta_info : dict
        Dictionary with meta information about the info log file.

    


In [6]:
time_traces, meta_info = load_cmrr_info(info_log)

Next load our ECG data using the corresponding `niphlem.input_data.load_cmrr_data` function. We specify the type of signal as an input. 

In [11]:
from niphlem.input_data import load_cmrr_data

print(load_cmrr_data.__doc__)



    Parameters
    ----------
    filename : str, pathlike
        Path to recording log file..
    sig_type : str
        Type of signal for use in dictionary
    info_dict : dict
        Dictionary with the meta information of the Info log file. It needs
        to be compute before by using the function load_cmrr_info.
    sync_scan : bool, optional
        Whether we want to resample the signal to be synchronized
        with the scanner times. The default is True.

    Returns
    -------
    signal : ndarray
        The recording signal, where the number of columns corresponds
        to the number of channels (ECG: 4, PULS: 1, RESP: 1) and the rows to
        observations.
    info_dict : dict
        Updated meta info of the physiological recording.

    


In [12]:
ecg_signal, meta_info = load_cmrr_data(ecg_log, info_dict=meta_info, sig_type="ECG")

### Run QC

To run QC on our ECG data, we use the `niphlem.report.make_ecg_report` function. 

In [13]:
from niphlem.report import make_ecg_report

print(make_ecg_report.__doc__)


    Parameters
    ----------
    ecg_signal : array-like of shape (n_physio_samples, n_channels)
        ECG Signal, where each column corresponds to a recording.
    fs : float
        Sampling frequency of ECG recording.
    delta: float
        minimum separation (in physio recording units) between
        events in signal to be considered peaks
    peak_rise: float
        relative height with respect to the 20th tallest events in signal
        to consider events as peak. The default is 0.75.
    ground : integer, optional
        Column in the input signal to be considered as a ground channel.
        This signal will be then substracted from the other channels.
        The default is None.
    high_pass : float, optional
        High-pass filtering frequency (in Hz). Only if filtering option
        is not None. The default is 0.6.
    low_pass : float, optional
        Low-pass filtering frequency (in Hz). Only if filtering option
        is not None. The default is 5.0.
    

As we can see, this function takes a number of inputs:

* our ECG signal
* sampling frequency of data signal
* minimum separation between signal events
* relative height of signal events (default 0.75)
* ground signal channel (optional, default None)
* high-pass filter frequency (default 0.6 Hz)
* low-pass filter frequency (default 5 Hz)
* output filepath (optional, default None)

Let's set these input values:

In [44]:
fs = 400
delta = 200
peak_rise = 0.95
ground = 3
high_pass = 0.5
low_pass = 15.0
outpath = "./"

Now we can run the function. There are three outputs:

1. The HTML report object, which contains the output of the quality control in html form (including graphs and statistics). If outpath is provided, this will be saved as ecg_qc.html *ecg_qc.html*.
2. An array with he timeseries of the average filtered ECG signal. If outpath is provided, this will be saved as *mean_filtered_signal.txt*.
3. An array that contains the onset times of each detected peak of the signal - in the case of ECG, this is the R component of the QRS waveform. If outpath is provided, this will be saved as *corrected_peaks.txt*.

In [45]:
ecg_report, mean_signal_filt, corrected_peaks = make_ecg_report(ecg_signal, 
                                                                fs=fs, 
                                                                delta=delta, 
                                                                ground=ground, 
                                                                high_pass=high_pass, 
                                                                low_pass=low_pass, 
                                                                outpath=outpath)

QC report for ECG signal saved in: /home/javi/Documentos/niphlem/examples/ecg_qc.html


We can always inspect directly the report on the jupyter-notebook server by just calling it

In [49]:
ecg_report

In [58]:
# or alternatively

from IPython.display import IFrame

IFrame(src='./ecg_qc.html', width=1000, height=600)

Finally, we can always save this report afterwards, without passing a path, by using its method `.save_as_html`

In [52]:
ecg_report.save_as_html("./the_same_qc_report.html")

In [57]:
IFrame(src='./the_same_qc_report.html', width=1000, height=600)